# Classroom Assignment: Case Study for Classroom Allocation

## Phase 2: Allocation of Classes in Classroom

Objective:
- Maximize the number of allocated rooms and the number of weak constraints met

## Mathematical Formulation

### Notation

- **$S$**: Set of available classrooms (eq. \ref{eq:conj_salas})  
- **$SQN$**: Set of classrooms that have a blackboard (eq. \ref{eq:conj_sala_quadroNegro})  
- **$SEC$**: Set of sessions that require classroom allocation (eq. \ref{eq:conj_sessoes})  
- **$SecTipo_{sec}$**: Set containing the types of classrooms required by the session (eq. \ref{eq:conj_sessao_tipo_sala})  
- **$SecCal$**: Set of sessions containing freshman classes (eq. \ref{eq:conj_sec_calouro})  
- **$SecQN$**: Set of sessions with a blackboard resource restriction (eq. \ref{eq:conj_sessao_quadro})  
- **$Sec_d$**: Subset of sessions that take place on weekday $d$ (eq. \ref{eq:conj_Sec_d})  
- **$Sec_h$**: Subset of sessions that take place at time $h$ (eq. \ref{eq:conj_Sec_h})  


### Variables

Considering

$$
s \in S, sec \in SEC, (d,h)\in TS_{tid}, tid = \pi_{tid}(sec)
$$

1. Decision variable $X_{s,tid,d,h}$ that determines the allocation of teacher/subject (session) in a classroom:

$$
X_{s,tid,d,h} \in (0,1)  =
\begin{cases}
1 &  \quad \text{if room $s$ is allocated to session $tid$ for the timeslot $(d,h) \in TS_{tid}$}\\ 
0 & \quad \text{otherwise} 
\end{cases}
$$

2. Slack variable $CapDiff_{s,sec}$ that ensures the allocation of the class is as close as possible to the room capacity (minimizing the difference between capacities):

$$
CapDiff_{s,sec} \in \mathbb{Z}_{+}
$$ 

3. Slack variable $TolVar$ for tolerance, allowing a small margin of error in the constraints:

$$
TolVar \in \mathbb{R}_{+}
$$ 

It relates to the constraint

$$
TolVar \leq 1 \times 10^{-6}
$$ 

> The variable $TolVar$ was adopted due to the difficulty of ensuring that the allocated room has the smallest possible difference between the requested and available capacity. The difference between capacities (required and allocated) is an integer value, but in practice, an infeasibility problem occurred due to numerical errors. Based on experiments, a tolerance of \( 1 \times 10^{-6} \) was defined, ensuring that rooms were allocated with the smallest possible capacity difference.

### Coefficients

The coefficients are values that act as weights in the objective function variables. These weights were defined experimentally. Priority is given to allocating a room belonging to the same institute responsible for the session. Thus, if the session $(sec)$ is associated with the same institute responsible for the room $(s)$, the weight will be 100. Otherwise, the standard coefficient value of 10 is applied.

$$
f_{s,sec} = 
\begin{cases}
100 &  \quad \text{if } SecIR_{sec} = IR_{s}\\ 
10 & \quad \text{otherwise}
\end{cases}
$$

### Objective Function

Maximize the number of weak constraints met, penalizing cases where the room allocation is not done in the most appropriate way. Thus, for each room allocated to a session that exceeds the required number of people, there will be a decrease in the objective function value. Since the function seeks maximization, whenever possible, this weak constraint will be met.

$$
MAX \sum_{s \in S}\sum_{sec \in SEC}\sum_{(d,h) \in TS_{tid}} f_{s,tid,d,h}X_{s,tid,d,h} - \sum_{s \in S}\sum_{sec \in SEC} CapDiff_{s,sec} * X_{s,tid,d,h}
$$

## Constraints

### Weak Constraints

**RNG1 (General Business Rule 1)**: Ensures that the required capacity for the session is allocated as close as possible to the actual room capacity, ensuring that the space is used as efficiently as possible.

$$
\sum_{s \in S} \sum_{sec \in SEC} CapDiff_{s,sec} \leq Cap_{s} - Vag_{sec} * X_{s,tid,d,h} + TolVar
$$

> The logic of function describes that the capacity should be close, assuming a constant that will perform an error tolerance (\textit{TolVar}). The value of \textit{TolVar} is set by default to 1e-6.

### Hard Constraints

**RNG2**: A room $(s)$ can be allocated to at most one session $(sec)$ on the same day $(d)$ and time $(h)$.

$$
\sum_{s \in S}\sum_{sec \in (Sec_h \cap Sec_d)} X_{s,tid,d,h} \leq 1  \quad (d,h) \in TS_{tid}, tid = \pi_{tid}(sec)
$$

**RNG3**: A session $(sec)$ should have only one classroom $(s)$ assigned. Sessions, by definition, already include a specific day and time.

$$
\sum_{sec \in SEC}\sum_{s \in S} X_{s,tid,d,h} = 1  \quad (d,h) \in TS_{tid}, tid = \pi_{tid}(sec)
$$

**RNP1 (Specific Business Rule 1)**:
The type of room allocated for each session must correspond to the requested type (e.g., Theoretical, Practical). The order in which room types are allocated is not mandatory and may vary according to availability, as long as the total number of all requested types is met.

$$
\sum_{sec \in SEC}\sum_{s \in S} X_{s,tid,d,h} = |\{st \in SecTipo_{sec} \mid st = P\}| \quad \text{where } STipo_{s} = st, (d,h) \in TS_{tid}, tid = \pi_{tid}(sec)
$$

$$
\sum_{sec \in SEC}\sum_{s \in S} X_{s,tid,d,h} = |\{st \in SecTipo_{sec} \mid st = T\}| \quad \text{where } STipo_{s} = st, (d,h) \in TS_{tid}, tid = \pi_{tid}(sec)
$$

Fixing a specific session $sec$, the number of each required room type must be equal to the number of rooms being allocated. Equation describes that the number of practical (P) rooms should be the same as requested. Equation refers to the number of theoretical (T) rooms.

**RNP2**: If the session is for a freshman class, a specific room must be designated for theoretical classes (F3014).

$$
\sum_{sec \in SR_r} X_{\text{F3014},tid,d,h} = |\{st \in SecTipo_{sec} \mid st = T\}|, \quad \text{where } (d,h) \in TS_{tid}, tid = \pi_{tid}(sec)
$$

Since we are dealing with the allocation of a specific room (F3014), whose category is previously known as theoretical (T), the number of sessions allocated to theoretical rooms must correspond to the number of required sessions.

**RNP3**: If the session has a blackboard restriction, do not allocate it to a room that has this type of resource.

$$
\sum_{sec \in SecQN}\sum_{s \in SQN} X_{s,tid,d,h} = 0  \quad (d,h) \in TS_{tid}, tid = \pi_{tid}(sec)
$$



### Final Formulation

Combining the formulation, we obtain the mathematical modeling as per, expressed by equation. It is observed that, from the problem definition, the final modeling results in an MIQP model, as discussed in Section. The quadratic term appears in the objective function, characterizing it as a non-linear problem. The constraints are linear. Regarding the variables, there is a real tolerance slack variable ($TolVar$), while the others are integers, characterizing the problem as mixed.

$$
\begin{aligned}
& \text{MAX} \sum_{s \in S}\sum_{sec \in SEC}\sum_{d \in D_s}\sum_{h \in T_s} f_{s,tid,d,h}X_{s,tid,d,h} - \sum_{s \in S}\sum_{sec \in SEC} CapDiff_{s,sec} * X_{s,tid,d,h}\\ \\
& \text{Subject to:} \\
& \sum_{s \in S} \sum_{sec \in SEC} CapDiff_{s,sec} \leq Cap_{s} - Vag_{sec} * X_{s,tid,d,h} + TolVar, \quad TolVar \leq 1 \times 10^{-6} \\
& \sum_{s \in S}\sum_{sec \in (Sec_h \cap Sec_d)} X_{s,tid,d,h} \leq 1 \\
& \sum_{sec \in SEC}\sum_{s \in S} X_{s,tid,d,h} = 1  \\
& \sum_{sec \in SEC}\sum_{s \in S} X_{s,tid,d,h} = |\{st \in SecTipo_{sec} \mid st = P\}| \quad \text{where } STipo_{s} = st \\
& \sum_{sec \in SEC}\sum_{s \in S} X_{s,tid,d,h} = |\{st \in SecTipo_{sec} \mid st = T\}| \quad \text{where } STipo_{s} = st \\
& \sum_{sec \in SR_r} X_{\text{F3014},tid,d,h} = |\{st \in SecTipo_{sec} \mid st = T\}| \\
& \sum_{sec \in SecQN}\sum_{s \in SQN} X_{s,tid,d,h} = 0  \\
& f_{s,sec} = 
\begin{cases}
100 &  \quad \text{if } SecIR_{sec} = IR_{s}\\ 
10 & \quad \text{otherwise}
\end{cases} \\
&  \quad X_{s,tid,d,h} \in \{0, 1\}, \quad \forall s \in S, sec \in SEC, (d,h) \in TS_{tid}, tid = \pi_{tid}(sec)
\end{aligned}
$$